In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
import pickle
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import logging
from open_spiel.python.examples.ubc_mccfr_cpp_example import action_to_bids
from open_spiel.python.examples.ubc_nfsp_example import policy_from_checkpoint
from open_spiel.python.examples.ubc_br import BR_DIR, make_dqn_agent
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.examples.ubc_decorators import CachingAgentDecorator

from open_spiel.python.pytorch.ubc_nfsp import NFSP
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap

import yaml

output_notebook()

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [10]:
# Why is best response not at least discovering that it can get 0 reward?

In [11]:
experiment_dir = '/shared/outputs/jan2_big_game_2/lstm_deep'
checkpoint_name = 'checkpoint_3000000'
br_player = 0
report_freq = 5_000

In [12]:
checkpoint_dir = os.path.join(experiment_dir, BR_DIR)
env_and_model = policy_from_checkpoint(experiment_dir, checkpoint_suffix=checkpoint_name)
game, policy, env, trained_agents, game_config = env_and_model.game, env_and_model.nfsp_policies, env_and_model.env, env_and_model.agents, env_and_model.game_config

## HACK IN NEW FEATURES
game = smart_load_sequential_game('clock_auction', game_parameters=dict(filename='big_game_2.json', handcrafted=True))

config_path = f'/apps/open_spiel/open_spiel/python/examples/network_configs/config_example_mlp.yml'

with open(config_path, 'rb') as fh:
    config = yaml.load(fh, Loader=yaml.FullLoader)
    config['num_training_episodes'] = 1_000_000
    config['rl_learning_rate'] = 0.01
    config['update_target_network_every'] = 10_000
num_training_episodes = config['num_training_episodes']

In [13]:
agents = []
for i in range(game.num_players()):
    if i == br_player:
        agents.append(make_dqn_agent(i, config, env, game, game_config))
    else:
        agent = trained_agents[i]
        agent = CachingAgentDecorator(agent)
        agents.append(agent)

logging.info(f"Training for {num_training_episodes} episodes")
# TRAINING PHASE

In [ ]:
episode_lengths = []
for i in range(num_training_episodes):
    if i % report_freq == 0:
        print(f"----Episode {i} ---")
        print(pd.Series(episode_lengths).describe())
        logging.info(f"----Episode {i} ---")
        if i > 10_000:
            loss = agents[br_player].loss
            logging.info(f"[P{br_player}] Loss: {loss}")
            print(agents[br_player].loss)
            print(check_on_q_values(agents[br_player], game))

    time_step = env.reset()
    episode_length = 0

    while not time_step.last():
        episode_length += 1
        player_id = time_step.observations["current_player"]
        agent = agents[player_id]
        agent_output = agent.step(time_step, is_evaluation=player_id != br_player)
        action_list = [agent_output.action]
        time_step = env.step(action_list)

    episode_lengths.append(episode_length)
    # Episode is over, step all agents with final info state.
    for player_id, agent in enumerate(agents):
        agent.step(time_step, is_evaluation=player_id != br_player)



----Episode 0 ---
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
dtype: float64


/apps/open_spiel/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [ ]:
# # fill buffer with some data with opponent dropping out
# # for action in range(12):
# #     for _ in range(10):
# #         time_step_1 = env.reset()
# #         env.step([action])
# #         time_step_2 = env.step([0])

# #         agents[br_player].add_transition(
# #             time_step_1, action, time_step_2
# #         )

# for i in tqdm(range(0)):
#     time_step = env.reset()
#     while not time_step.last():
#         player_id = time_step.observations["current_player"]
#         agent = agents[player_id]
#         legal_actions = time_step.observations["legal_actions"][player_id]
#         action_list = [np.random.choice(legal_actions)]
#         time_step = env.step(action_list)
#         if player_id == br_player:
#             agent._prev_timestep = time_step
#             agent._prev_action = action

#     # Episode is over, step all agents with final info state.
#     for player_id, agent in enumerate(agents):
#         agent.step(time_step, is_evaluation=player_id != br_player)

# # # switch learning settings temporarily
# # min_buffer_size = agents[br_player]._min_buffer_size_to_learn
# # batch_size = agents[br_player]._batch_size
# # agents[br_player]._min_buffer_size_to_learn = 1
# # agents[br_player]._batch_size = 12

# # train a bunch
# for iteration in tqdm(range(1000)):
#     agents[br_player].learn()
    
# # # change settings back
# # agents[br_player]._min_buffer_size_to_learn = min_buffer_size
# # agents[br_player]._batch_size = batch_size

# # clear buffer
# agents[br_player]._replay_buffer.clear()
# agents[br_player]._target_q_network.load_state_dict(agents[br_player]._q_network.state_dict())

In [ ]:
import pickle
with open("stupid.pkl", 'wb') as f:
    pickle.dump(agents[br_player]._q_network.state_dict(), f)

In [ ]:
agent.mapRange(4477.8413)

In [ ]:
agents[0]._replay_buffer._data[:10]

In [ ]:
pd.Series(episode_lengths[-10000:]).describe()

In [2]:
from open_spiel.python.examples.ubc_utils import *
print("HI")
smart_load_sequential_game('clock_auction', game_parameters=dict(filename='big_game_2.json', handcrafted=True)).new_initial_state().child(0).child(0).child(11).child(11).information_state_tensor()

HI


[1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 -0.25,
 19.5,
 19.25,
 29.25,
 29.0,
 48.75,
 48.5,
 58.5,
 58.25,
 78.0,
 77.75,
 2.0,
 90.75,
 60.5,
 30.25,
 2.0,
 1.0,
 1.0,
 4.0,
 2.0,
 2.0]

In [3]:
game = smart_load_sequential_game('clock_auction', game_parameters=dict(filename='big_game_2.json', handcrafted=True))


In [4]:
dir(game)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'action_to_string',
 'deserialize_state',
 'get_parameters',
 'get_type',
 'information_state_tensor_layout',
 'information_state_tensor_shape',
 'information_state_tensor_size',
 'make_observer',
 'max_chance_nodes_in_history',
 'max_chance_outcomes',
 'max_game_length',
 'max_history_length',
 'max_move_number',
 'max_utility',
 'min_utility',
 'new_initial_state',
 'new_initial_state_for_population',
 'new_initial_states',
 'num_distinct_actions',
 'num_players',
 'observation_tensor_layout',
 'observation_tensor_shape',
 'observation_tensor_size',
 'policy_tensor_shape',
 'utility_sum']

In [10]:
game.get_parameters()['game']['name']

'clock_auction'